In [1]:
import argparse
import os
import glob
import sys
import torch
import numpy as np
import random
import pandas as pd
from datetime import datetime
from dateutil import tz
from torch.nn import CrossEntropyLoss
import args_cnn as args
import torch.optim as optim
from tqdm import tqdm
import config
from loss import CCCLoss
from utils import Logger, seed_worker
from train import train_model
from eval import  calc_ccc
from model import Model
from dataset import MuSeDataset
from data_parser import get_data_partition, segment_sample, normalize_data
import pickle

In [2]:
args.log_file_name = '{}_[{}]_[{}]_[{}_{}_{}]_[{}_{}]'.format(
    datetime.now(tz=tz.gettz()).strftime("%Y-%m-%d-%H-%M"), '_'.join(args.feature_set), args.emo_dim,
    args.d_rnn, args.rnn_n_layers, args.rnn_bi, args.lr, args.batch_size)

# adjust your paths in config.py
args.paths = {'log': os.path.join(config.LOG_FOLDER, args.task),
              'data': os.path.join(config.DATA_FOLDER, args.task),
              'model': os.path.join(config.MODEL_FOLDER, args.task, args.log_file_name)}
if args.predict:
    args.paths['predict'] = os.path.join(config.PREDICTION_FOLDER, args.task, args.log_file_name)
if args.save:
    args.paths['save'] = os.path.join(args.save_path, args.task, args.log_file_name)
for folder in args.paths.values():
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)

args.paths.update({'features': config.PATH_TO_ALIGNED_FEATURES[args.task],
                   'labels': config.PATH_TO_LABELS[args.task],
                   'partition': config.PARTITION_FILES[args.task]})

# sys.stdout = Logger(os.path.join(args.paths['log'], args.log_file_name + '.txt'))
# print(' '.join(sys.argv))

In [3]:
### load data

task = args.task
paths = args.paths
feature_set = args.feature_set
emo_dim = args.emo_dim
normalize = args.normalize
norm_opts = args.norm_opts
win_len = args.win_len
hop_len = args.hop_len
save = args.save
apply_segmentation = args.apply_segmentation


In [4]:
feature_path = paths['features']
label_path = paths['labels']

data_file_name = f'data_{task}_{"_".join(feature_set)}_{emo_dim}_{"norm_" if normalize else ""}{win_len}_' \
    f'{hop_len}{"_seg" if apply_segmentation else ""}.pkl'
data_file = os.path.join(paths['data'], data_file_name)

# if os.path.exists(data_file):  # check if file of preprocessed data exists
#     print(f'Find cached data "{os.path.basename(data_file)}".')
#     data = pickle.load(open(data_file, 'rb'))
#     return data

print('Constructing data from scratch ...')
data = {'train': {'feature': [], 'label': [], 'meta': []},
        'devel': {'feature': [], 'label': [], 'meta': []},
        'test': {'feature': [], 'label': [], 'meta': []}}
vid2partition, partition2vid = get_data_partition(paths['partition'])
feature_dims = [0] * len(feature_set)

feature_idx = 2  # first two columns are timestamp and segment_id, features start with the third column


Constructing data from scratch ...


In [5]:

for partition, vids in partition2vid.items():
    for vid in vids:
        sample_data = []
        segment_ids_per_step = []  # necessary for MuSe-Sent

        # parse features
        for i, feature in enumerate(feature_set):
            #print(feature)
            feature_file = os.path.join(feature_path, feature, vid + '.csv')
            assert os.path.exists(
                feature_file), f'Error: no available "{feature}" feature file for video "{vid}": "{feature_file}".'
            df = pd.read_csv(feature_file)
            feature_dims[i] = df.shape[1] - feature_idx
            if i == 0:
                feature_data = df  # keep timestamp and segment id in 1st feature val
                segment_ids_per_step = df.iloc[:, 1]
            else:
                feature_data = df.iloc[:, feature_idx:]
            sample_data.append(feature_data)
        data[partition]['feature_dims'] = feature_dims

        # parse labels
        label_file = os.path.join(label_path, emo_dim, vid + '.csv')
        assert os.path.exists(
            label_file), f'Error: no available "{emo_dim}" label file for video "{vid}": "{label_file}".'
        df = pd.read_csv(label_file)

        if task == 'sent':
            label = df['class_id'].values
            label_stretched = [label[s_id - 1] if not pd.isna(s_id) else pd.NA for s_id in segment_ids_per_step]
            label_data = pd.DataFrame(data=label_stretched, columns=[emo_dim])
        else:
            label_data = pd.DataFrame(data=df['value'].values, columns=[emo_dim])
        sample_data.append(label_data)

        # concat
        sample_data = pd.concat(sample_data, axis=1)
        if partition != 'test':
            sample_data = sample_data.dropna()

        # segment
        if apply_segmentation:
            if task == 'sent':
                seg_type = 'by_segs_only' if partition != 'train' else 'by_segs'
                samples = segment_sample(sample_data, win_len, hop_len, seg_type)
            elif task in ['wilder', 'physio', 'stress']:
                if partition == 'train':
                    samples = segment_sample(sample_data, win_len, hop_len, 'normal')
                else:
                    samples = [sample_data]
        else:
            if task == 'sent':
                samples = segment_sample(sample_data, win_len, hop_len, 'by_segs_only')
            else:
                samples = [sample_data]

        # store
        for i, segment in enumerate(samples):  # each segment has columns: timestamp, segment_id, features, labels
            n_emo_dims = 1
            if len(segment.iloc[:, feature_idx:-n_emo_dims].values) > 0:  # check if there are features
                meta = np.column_stack((np.array([int(vid)] * len(segment)),
                                        segment.iloc[:, :feature_idx].values))  # video_id, timestamp, segment_id
                data[partition]['meta'].append(meta)
                data[partition]['label'].append(segment.iloc[:, -n_emo_dims:].values)
                data[partition]['feature'].append(segment.iloc[:, feature_idx:-n_emo_dims].values)

if normalize:
    idx_list = []

    assert norm_opts is not None and len(norm_opts) == len(feature_set)
    norm_opts = [True if norm_opt == 'y' else False for norm_opt in norm_opts]

    print(f'Feature dims: {feature_dims} ({feature_set})')
    feature_dims = np.cumsum(feature_dims).tolist()
    feature_dims = [0] + feature_dims

    norm_feature_set = []  # normalize data per feature and only if norm_opts is True
    for i, (s_idx, e_idx) in enumerate(zip(feature_dims[0:-1], feature_dims[1:])):
        norm_opt, feature = norm_opts[i], feature_set[i]
        if norm_opt:
            norm_feature_set.append(feature)
            idx_list.append([s_idx, e_idx])

    print(f'Normalized features: {norm_feature_set}')
    data = normalize_data(data, idx_list)

if save:  # save loaded and preprocessed data
    print('Saving data...')
    pickle.dump(data, open(data_file, 'wb'))


Saving data...


In [6]:
data_loader = {}
for partition in data.keys():  # one DataLoader for each partition
    set = MuSeDataset(data, partition)
    batch_size = args.batch_size if partition == 'train' else 1
    shuffle = True if partition == 'train' else False  # shuffle only for train partition
    data_loader[partition] = torch.utils.data.DataLoader(set, batch_size=batch_size, shuffle=shuffle, num_workers=4,
                                                         worker_init_fn=seed_worker)

In [7]:
args.d_in = data_loader['train'].dataset.get_feature_dim()
args.n_targets = 1
criterion = CCCLoss()
score_str = 'CCC'


In [8]:
import torch.nn as nn
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence

In [9]:
class RNN(nn.Module):
    def __init__(self, d_in, d_out, n_layers=1, bi=True, dropout=0):
        super(RNN, self).__init__()
        self.rnn = nn.LSTM(input_size=d_in, hidden_size=d_out, bidirectional=bi, num_layers=n_layers, dropout=dropout)

    def forward(self, x, x_len):
        
        x = x.permute(1,0,2)
        
        x_packed = pack_padded_sequence(x, x_len.cpu(), batch_first=True, enforce_sorted=False)
        x_out, (hidden, cell) = self.rnn(x_packed)
        x_padded = pad_packed_sequence(x_out, total_length=x.size(1), batch_first=True)[0]
        x_padded = x_padded.permute(1,0,2)
        return x_padded, hidden, cell



class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(Encoder, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.rnn = RNN(input_size, hidden_size, num_layers, bi=True, dropout=dropout)
        
        self.fc_hidden = nn.Linear(hidden_size*2, hidden_size)
        self.fc_cell = nn.Linear(hidden_size*2, hidden_size)
        
    def forward(self, x, x_len):
        
        N, seq_length, num_feats = x.shape
        x = x.reshape((seq_length, N, num_feats))
        outputs, hidden, cell = self.rnn(x, x_len)
        hidden = self.fc_hidden(torch.cat((hidden[0:1], hidden[1:2]), dim=2))
        cell = self.fc_cell(torch.cat((cell[0:1], cell[1:2]), dim=2))
        return outputs, hidden, cell

class OutLayer(nn.Module):
    def __init__(self, d_in, d_hidden, d_out, dropout=.0, bias=.0):
        super(OutLayer, self).__init__()
        self.fc_1 = nn.Sequential(nn.Linear(d_in, d_out), nn.ReLU(True), nn.Dropout(dropout))
        #self.fc_2 = nn.Linear(d_hidden, d_out)
        #nn.init.constant_(self.fc_2.bias.data, bias)

    def forward(self, x):
        y = self.fc_1(x)
        return y

class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout):
        super(Decoder, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.rnn = nn.LSTM(hidden_size*2 + input_size, hidden_size, num_layers, dropout=dropout)
        self.energy = nn.Linear(hidden_size*3,1)
        self.softmax = nn.Softmax(dim=0)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x, x_len, encoder_states, hidden, cell):
        N, seq_length, num_feats = x.shape
        x = x.reshape((seq_length, N, num_feats))
        x = self.dropout(x)
        
        sequence_length = encoder_states.shape[0]
        h_reshaped = hidden.repeat(sequence_length, 1, 1)
        energy = self.relu(self.energy(torch.cat((h_reshaped, encoder_states), dim = 2)))
        attention = self.softmax(energy)
        
        #(seq_length, N, 1)
        attention = attention.permute(1,2,0)
        #(N, 1, seq_length)
        encoder_states = encoder_states.permute(1,0,2)
        
        
        context_vector = torch.bmm(attention, encoder_states).permute(1,0,2)
        context_vector = context_vector.repeat(sequence_length, 1, 1)
        rnn_input = torch.cat((context_vector, x), dim=2)
        #print(rnn_input.shape)
        outputs, (hidden, cell) = self.rnn(rnn_input, (hidden, cell))
        outputs = outputs.permute(1,0,2)
        #print(outputs.shape)

        return outputs, hidden, cell
    

In [10]:
class Model(nn.Module):
    def __init__(self, encoder, decoder, out):
        super(Model, self).__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.out = out

#         self.input_size = input_size
#         self.hidden_size = hidden_size
#         self.output_size = output_size
#         self.num_layers = num_layers
#         self.fc_hidden = fc_hidden
#         self.dropout = dropout
#         self.encoder = Encoder(input_size, hidden_size, num_layers, dropout)
#         self.decoder = Decoder(input_size, hidden_size, output_size, num_layers, dropout)
#         self.out = OutLayer(d_in = hidden_size, d_hidden = fc_hidden, d_out = output_size)
        
    def forward(self, x, x_len):
        encoder_states, hidden, cell = self.encoder(x, x_len)
        outputs, hidden, cell = self.decoder(x, x_len, encoder_states, hidden, cell)
        outputs = self.out(outputs)
        return outputs
        

In [11]:
enc = Encoder(input_size = 88, hidden_size=32, num_layers=1, dropout = 0)
dec = Decoder(input_size=88, hidden_size=32, output_size=1, num_layers=1, dropout=0)
outl = OutLayer(d_in = 32, d_hidden = 8, d_out = 1)
model = Model(enc, dec, outl)

In [12]:
lr = args.lr
regularization = args.regularization
train_loader, val_loader, test_loader = data_loader['train'], data_loader['devel'], data_loader['test']


In [13]:
train_loader, val_loader, test_loader = data_loader['train'], data_loader['devel'], data_loader['test']

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=regularization)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', patience=5, factor=0.5,
                                                    min_lr=1e-5, verbose=True)
metric = 'Macro-F1' if task == 'sent' else 'CCC'
best_val_loss = float('inf')
best_val_score = -1
best_model_file = ''
early_stop = 0

In [14]:

def train(task, model, train_loader, epoch, optimizer, criterion, use_gpu=False):
    start_time = time.time()
    report_loss, report_size = 0, 0
    total_loss, total_size = 0, 0

    model.train()
    for batch, batch_data in enumerate(train_loader, 1):
        features, feature_lens, labels, metas = batch_data
        batch_size = features.size(0)

        if use_gpu:
            model.cuda()
            features = features.cuda()
            feature_lens = feature_lens.cuda()
            labels = labels.cuda()

        optimizer.zero_grad()

        preds = model(features, feature_lens)

        loss = criterion(preds[:, :, 0], labels[:, :, 0], feature_lens)

        loss.backward()
        optimizer.step()

        report_loss += loss.item() * batch_size
        report_size += batch_size

        avg_loss = report_loss / report_size
        elapsed_time = time.time() - start_time
        print(
            f"Epoch:{epoch:>3} | Batch: {batch:>3} | Lr: {optimizer.state_dict()['param_groups'][0]['lr']:>1.5f}"
            f" | Time used(s): {elapsed_time:>.1f} | Training loss: {avg_loss:>.4f}")

        total_loss += report_loss
        total_size += report_size
        report_loss, report_size, start_time = 0, 0, time.time()

    train_loss = total_loss / total_size
    return train_loss


In [15]:
def evaluate(task, model, data_loader, criterion, use_gpu=False, predict=False, prediction_path=''):
    losses, sizes = 0, 0
    full_preds = []
    if predict:
        full_metas = []
    else:
        full_labels = []

    if task == 'sent':
        full_logits = []
        full_metas_stepwise = []

    model.eval()
    with torch.no_grad():
        for batch, batch_data in enumerate(data_loader, 1):
            features, feature_lens, labels, metas = batch_data
            batch_size = features.size(0)

            if use_gpu:
                model.cuda()
                features = features.cuda()
                feature_lens = feature_lens.cuda()
                labels = labels.cuda()

            preds = model(features, feature_lens)

            if predict:
                full_metas.append(metas.detach().squeeze(0).numpy())
                if task == 'sent':
                    full_metas_stepwise.append(metas_stepwise.detach().squeeze(0).numpy())
                    full_logits.append(logits_stepwise.cpu().detach().squeeze(0).numpy())
            else:
                loss = criterion(preds[:, :, 0], labels[:, :, 0], feature_lens)

                losses += loss.item() * batch_size
                sizes += batch_size

                full_labels.append(labels.cpu().detach().squeeze(0).numpy())
            full_preds.append(preds.cpu().detach().squeeze(0).numpy())

        if predict:
            write_predictions(full_metas, full_preds, prediction_path)
            return
        else:
            score = calc_ccc(full_preds, full_labels)
            total_loss = losses / sizes
            return total_loss, score


In [16]:
epochs = args.epochs
use_gpu = args.use_gpu
import time
current_seed = args.seed
seed = args.seed

for epoch in tqdm(range(1, epochs + 1)):
    train_loss = train(task, model, train_loader, epoch, optimizer, criterion, use_gpu)
    val_loss, val_score = evaluate(task, model, val_loader, criterion, use_gpu)

    print('-' * 50)
    print(f'Epoch:{epoch:>3} | [Train] | Loss: {train_loss:>.4f}')
    print(f'Epoch:{epoch:>3} |   [Val] | Loss: {val_loss:>.4f} | [{metric}]: {val_score:>7.4f}')
    print('-' * 50)

    if val_score > best_val_score:
        early_stop = 0
        best_val_score = val_score
        best_val_loss = val_loss
        #best_model_file = save_model(model, model_path, current_seed)

    else:
        early_stop += 1
        if early_stop >= 15:
            print(f'Note: target can not be optimized for 15 consecutive epochs, early stop the training process!')
            print('-' * 50)
            break

    lr_scheduler.step(1 - np.mean(val_score))

print(f'Seed {current_seed} | '
      f'Best [Val {metric}]:{best_val_score:>7.4f} | Loss: {best_val_loss:>.4f}')


  0%|                                                                                          | 0/100 [00:05<?, ?it/s]


RuntimeError: DataLoader worker (pid(s) 6520, 12636, 28912, 32748) exited unexpectedly